In [1]:
# dependencies 
import pandas as pd
import random as rd
import numpy as np
import matplotlib.pyplot as plt
import timeit
import time
import requests
import gmaps
import json
#from pandas.io.json import json_normalize 

# libraries and modules
from datetime import datetime
from citipy import citipy
from scipy.stats import linregress
from bs4 import BeautifulSoup

# api keys
from config import weather_api_key
from config import g_key

In [2]:
# create a new set of 2,000 random latitudes and longitudes.

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# zip into tuples and unzip into list
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

# create list of cities from coordinates
cities = []
for c in coordinates:
    city = citipy.nearest_city(c[0], c[1]).city_name

    if city not in cities:
        cities.append(city)
len(cities)

# perform an api call with Open Weather Map and retrieve
# URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list. -- LEARN ENUMERATE!
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# lat long max temp humidity cloudiness windspeed weather descripton

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # always note if its a list or dictionary
        weather_desc = city_weather["weather"][0]["description"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description": weather_desc,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

#Re-order 
new_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Weather Description"]
city_data_df = city_data_df[new_order]
city_data_df.head(30)

# output dataframe and CSV


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tiksi
Processing Record 2 of Set 1 | ballybofey
Processing Record 3 of Set 1 | arraial do cabo
Processing Record 4 of Set 1 | clyde river
Processing Record 5 of Set 1 | victoria
Processing Record 6 of Set 1 | kaitangata
Processing Record 7 of Set 1 | ancud
Processing Record 8 of Set 1 | mar del plata
Processing Record 9 of Set 1 | puerto ayora
Processing Record 10 of Set 1 | russell
Processing Record 11 of Set 1 | tuktoyaktuk
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | barrow
Processing Record 14 of Set 1 | carnduff
Processing Record 15 of Set 1 | saint-philippe
Processing Record 16 of Set 1 | mineral wells
Processing Record 17 of Set 1 | vinni
Processing Record 18 of Set 1 | malanje
Processing Record 19 of Set 1 | mount gambier
Processing Record 20 of Set 1 | thompson
Processing Record 21 of Set 1 | ushuaia
Processing Record 22 of Set 1 | longyearbyen
Processing Recor

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Tiksi,RU,2021-10-01 01:11:33,71.6872,128.8694,30.96,68,100,4.18,overcast clouds
1,Ballybofey,IE,2021-10-01 01:11:34,54.8000,-7.7833,52.03,94,90,10.13,overcast clouds
2,Arraial Do Cabo,BR,2021-10-01 01:01:22,-22.9661,-42.0278,69.53,87,100,8.57,overcast clouds
3,Clyde River,CA,2021-10-01 01:11:34,70.4692,-68.5914,33.31,74,100,6.69,overcast clouds
4,Victoria,HK,2021-10-01 01:09:10,22.2855,114.1577,90.25,82,0,1.01,clear sky
5,Kaitangata,NZ,2021-10-01 01:04:18,-46.2817,169.8464,56.93,52,68,12.28,broken clouds
6,Ancud,CL,2021-10-01 01:11:35,-41.8697,-73.8203,42.33,64,100,5.30,overcast clouds
7,Mar Del Plata,AR,2021-10-01 01:10:16,-38.0023,-57.5575,56.88,63,0,4.00,fog
8,Puerto Ayora,EC,2021-10-01 01:04:00,-0.7393,-90.3518,70.16,91,54,6.73,broken clouds
9,Russell,US,2021-10-01 01:11:36,32.3502,-85.1999,73.96,89,75,0.00,broken clouds


In [3]:
city_data_df.to_csv("WeatherPy_Database.csv")